In [1]:
from jenga.tasks.reviews import VideogameReviewsTask
from jenga.evaluation.schema_stresstest import SchemaStresstest

import tensorflow_data_validation as tfdv

import numpy as np
import pandas as pd

### Instantiate the video game reviews task with a randomly chosen seed

In [2]:
seed = np.random.randint(2**32 - 1)

task = VideogameReviewsTask(seed=seed)

### Create a tfdv schema by first auto-infering it from training data and then adjusting it

In [3]:
train_data_stats = tfdv.generate_statistics_from_dataframe(task.train_data)
schema = tfdv.infer_schema(statistics=train_data_stats)
review_date_feature = tfdv.get_feature(schema, 'review_date')
review_date_feature.distribution_constraints.min_domain_mass = 0.0

In [4]:
schema

feature {
  name: "marketplace"
  type: BYTES
  domain: "marketplace"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "customer_id"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "review_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_parent"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_title"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_category"
  type: BYT

### Train the baseline model for the task

In [5]:
model = task.fit_baseline_model()

### Run a stress test for the schema and the model with 250 randomly chosen corruptions, and mark performance drops of 3% as failures

In [6]:
stress_test = SchemaStresstest()
results = stress_test.run(task, model, schema, num_corruptions=250, performance_threshold=.03)

MissingValues: {'column': 'verified_purchase', 'fraction': 0.57, 'sampling': 'MNAR', 'na_value': ''}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.53}
SwappedValues: {'column': 'review_headline', 'fraction': 0.08, 'sampling': 'CAR', 'swap_with': 'review_headline'}
MissingValues: {'column': 'vine', 'fraction': 0.64, 'sampling': 'MCAR', 'na_value': ''}
SwappedValues: {'column': 'verified_purchase', 'fraction': 0.75, 'sampling': 'CAR', 'swap_with': 'marketplace'}
MissingValues: {'column': 'title_and_review_text', 'fraction': 0.0, 'sampling': 'MNAR', 'na_value': ''}
MissingValues: {'column': 'customer_id', 'fraction': 0.94, 'sampling': 'MCAR', 'na_value': ''}
MissingValues: {'column': 'review_id', 'fraction': 0.53, 'sampling': 'MNAR', 'na_value': ''}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.74}
BrokenCharacters: {'column': 'vine', 'fraction': 0.89}
BrokenCharacters: {'column': 'vine', 'fraction': 0.83}
MissingValues: {'column': 'review_id', 'fraction': 0.46, 's

### Look at the dataframe containing the results

In [7]:
results

,corruption,status,anomalies,baseline_score,corrupted_score
0,"MissingValues: {'column': 'verified_purchase',...",FP,"{'verified_purchase': description: ""Examples c...",0.791584,0.773935
1,"BrokenCharacters: {'column': 'customer_id', 'f...",TN,{},0.791584,0.791584
2,"SwappedValues: {'column': 'review_headline', '...",TN,{},0.791584,0.791584
3,"MissingValues: {'column': 'vine', 'fraction': ...",TP,"{'vine': description: ""Examples contain values...",0.791584,0.764364
4,"SwappedValues: {'column': 'verified_purchase',...",FP,"{'verified_purchase': description: ""Examples c...",0.791584,0.777793
...,...,...,...,...,...
245,"MissingValues: {'column': 'star_rating', 'frac...",TP,"{'star_rating': description: ""The feature was ...",0.791584,NaN
246,"BrokenCharacters: {'column': 'vine', 'fraction...",TN,{},0.791584,0.791584
247,"Scaling: {'column': 'star_rating', 'fraction':...",FN,{},0.791584,0.608570
248,BrokenCharacters: {'column': 'review_headline'...,TN,{},0.791584,0.791584
